In [5]:
import nuclio 
import mlrun
import os

In [6]:
import storey

In [3]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [4]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames

In [5]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/mlrun:0.6.5"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [6]:
%nuclio cmd -c pip install v3io

In [7]:
# nuclio: start-code

In [7]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd
import numpy
import os
import math
import datetime

In [8]:
def rm_inf(data):
    print(data)
    data = [0 if item == math.inf or item == -math.inf else item for item in data ]
    return data

In [21]:
def handler(context, event):
    context.logger.info("Reading features from feature vector")
    # Reading the data from feature service
    start = datetime.datetime.now()-datetime.timedelta(5)
    end = datetime.datetime.now()
    df = fs.get_offline_features("stocks-vec",start_time = start, end_time = end,entity_timestamp_column="Datetime").to_dataframe()
#     df["Datetime"] = df["Datetime"].apply(lambda x:datetime.datetime.strftime(x,"%Y-%m-%d %H:%M:%S"))
    df = df.drop(["Content","Link"],axis = 1)
    data = df.fillna(0).values.tolist()
    
    columns = [{'text': key, 'type': 'object'} for key in df.columns]
    
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response

In [22]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project=context.PROJECT_NAME)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc', 'INTC' : 'intel-corp'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.*",
                "news.*"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.spec.with_indexes=True
    vector.save()

In [23]:
# nuclio: end-code

In [24]:
mlrun.set_environment(project="stocks-" + os.getenv('V3IO_USERNAME'))

('stocks-dani', 'v3io:///projects/{{run.project}}/artifacts')

In [25]:
# test local
init_context(context)
event = ""
df = handler(context,event)

Python> 2021-07-28 10:56:32,317 [info] Initalizing context & constructing featrure vector
Python> 2021-07-28 10:56:32,377 [info] Reading features from feature vector


In [36]:
# test remote deployment
from mlrun import code_to_function
import os
fn = code_to_function('vector_reader',
                      handler='handler')

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : "stocks-" + os.getenv('V3IO_USERNAME'),
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream'})
fn.spec.max_replicas = 2

In [37]:
addr = fn.deploy(project="stocks-" + os.getenv('V3IO_USERNAME'))

> 2021-07-28 11:07:25,960 [info] Starting remote function deploy
2021-07-28 11:07:26  (info) Deploying function
2021-07-28 11:07:26  (info) Building
2021-07-28 11:07:26  (info) Staging files and preparing base images
2021-07-28 11:07:26  (info) Building processor image
2021-07-28 11:07:34  (info) Build complete
2021-07-28 11:07:38  (info) Function deploy complete
> 2021-07-28 11:07:39,078 [info] function deployed, address=default-tenant.app.app-lab-testing.iguazio-cd2.com:30821


In [38]:
!curl {addr}